In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

# Image Captioning 문제 



In [1]:
import torch # torch library 
import torch.nn as nn # Nueral Network에 대한 package
import numpy as np  # numpy 
import torch.nn.functional as F # pytorch function 들을 사용하기 위한 용도 
import random


**문제 설명**

다음의 Attention을 구현하도록 합니다  (아래 그림 참조)
1. Dot 기반 
2. General 기반

위의 각각의 Attention을 수식을 보고 구현하면 됩니다.

주석으로 * TO DO * 처리 된 영역을 채워주세요 Input과 Return 의 크기는 주석으로 적어 두었습니다. 

코드 작성후 각 모듈의 Unit test code를 통과 했다면 코드는 올바르게 작동한 것입니다.




*   AttentionDot 클래스는 Dot 기반의 attention model 입니다.
    ( Line by line 으로 주석을 달아놓았으니, 참고해서 작성해주세요. )
    
    => AttentionDot 클래스가 올바로 작성되었다면, test_attention_dot 함수를 선언한 뒤 호출했을 때,
    PASS! Good job! 이라는 메시지가 출력됩니다.


*   Attention_General 클래스는 Dot product 이전에 linear layer를 하나 더 통과시킨 뒤  query와 key간의 내적을 하는 모듈입니다.
    위와 마찬가지로, 주석을 참고해서 작성해주세요.
    (주의: key에 대해서만 새로이 추가된 weight를 적용하고, value에 대해서는 weight를 적용하지 않습니다.)
    
    => Attention_General 클래스가 올바로 작성되었다면, PASS! Good job! 이라는 메시지가 출력됩니다.
    
    

![대체 텍스트](https://i.stack.imgur.com/tiQkz.png)

**1-1 : Attention  Dot-product Attention**


In [2]:
class AttentionDot(nn.Module):
    """
    Attention Network.
    """

    def __init__(self):
        
        super(AttentionDot, self).__init__()
        self.softmax = # *TO DO*  # softmax layer to calculate weights

    def forward(self, key, value, query):
        """
        Forward propagation.
        :param encoder_out: encoded features, a tensor of dimension (batch_size, num_pixels, hidden_state)
        :param decoder_hidden: previous decoder output, a tensor of dimension (batch_size, hidden_state)
        :return: attention weighted encoding, weights
        """
        transpose_encoder_out =  key.permute(0,2,1)
        logits = # *TO DO* # (batch_size,1, num_pixels) # hint: Batch Mat Mul
        logits = logits.squeeze(1)
        alpha = # *TO DO*  # (batch_size, num_pixels)
        
        attention_weighted_encoding = # *TO DO*  # (batch_size, encoder_dim)
        
        return attention_weighted_encoding, alpha


In [3]:
def test_attention_dot():
  model = AttentionDot()
  test_encoder_out = [
      torch.zeros((1,3,2)),
      torch.ones((1,3,2)),
      torch.zeros((1,3,2)),
      torch.FloatTensor([[[1,2],[3,4],[5,6]]]),
      torch.FloatTensor([[[6,5],[4,3],[2,1]]]),
      torch.FloatTensor([[[1,0],[1,0],[1,0]]]),
      torch.FloatTensor([[[0,1],[0,1],[0,1]]]),
      torch.FloatTensor([[[1,1],[1,1],[1,1]]])
  ]
  test_decoder_hidden = [
      torch.zeros((1,2)),
      torch.ones((1,2)),
      torch.ones((1,2)),
      torch.FloatTensor([[1,1]]),
      torch.FloatTensor([[2,3]]),
      torch.FloatTensor([[0,1]]),
      torch.FloatTensor([[0,1]]),
      torch.FloatTensor([[1,0]])
  ]
  
  test_dot = []
#   test_encoder_out = torch.zeros((1,3,10))
#   test_decoder_hidden = torch.zeros((1,10))
  for i in range(len(test_encoder_out)):
    attention_context, alpha = model(test_encoder_out[i], test_encoder_out[i], test_decoder_hidden[i])
    test_dot += [alpha]

  gt_alpha1 = [
      torch.zeros((1,3)).fill_(1/3),
      torch.zeros((1,3)).fill_(1/3),
      torch.zeros((1,3)).fill_(1/3),
      torch.FloatTensor([[3.2932e-04, 1.7980e-02, 9.8169e-01]]),
      torch.FloatTensor([[9.9995e-01, 4.5398e-05, 2.0611e-09]]),
      torch.zeros((1,3)).fill_(1/3),
      torch.zeros((1,3)).fill_(1/3),
      torch.zeros((1,3)).fill_(1/3)
  ]

  result = [torch.all(torch.lt(torch.abs(torch.add(test_dot[i], -gt_alpha1[i])), 1e-4)).item() for i in range(len(test_encoder_out))]

  
  assert (torch.FloatTensor(result).sum().item() == len(test_encoder_out)) == 1
  
  print ("PASS! Good job! ")

In [4]:
## Test code  Error 나면 코드 확인필요 "PASS! Good job" 이 아무문제 없이 출력시 성공 
test_attention_dot()

PASS! Good job! 


**1-2 Attention General Attention**

In [5]:
class Attention_General(nn.Module):
    """
    Attention Network.
    """

    def __init__(self, hidden_dim = 3):
        """
        :param hidden_dim: feature size of hidden dimension 
        """
        super(Attention_General, self).__init__()
        self.Wa = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.softmax = # *TO DO*  # softmax layer to calculate weights
        self._weight_init()
    def _weight_init(self):
        self.Wa.weight.data = torch.tensor([[1.0,1.0,0.0],[-1.0,1.0,0.0],[0.0,1.0,-1.0]])
    def forward(self, key, value, query):
        """
        Forward propagation.
        :param encoder_out: encoded features, a tensor of dimension (batch_size, num_pixels, hidden_dim)
        :param decoder_hidden: previous decoder output, a tensor of dimension (batch_size, hidden_dim)
        :return: attention weighted encoding, weights
        """
        x = # *TO DO*  # (batch_size, num_pixels, hidden_dim)
        
        transpose_x_out =  x.permute(0,2,1)
        logits = # *TO DO* # (batch_size,1, hidden_dim) # hint
        logits = logits.squeeze(1)
        alpha = # *TO DO*  # (batch_size, num_pixels)
        
        attention_weighted_encoding = # *TO DO*  # (batch_size, encoder_dim)

        return attention_weighted_encoding, alpha


#### Test 코드

In [6]:
def test_attention_general():
  model = Attention_General()
  test_encoder_out = torch.zeros((3,3,3))
  test_decoder_hidden = torch.zeros((3,3))
  attention_context, alpha = model(test_encoder_out, test_encoder_out, test_decoder_hidden) 
  gt_alpha1 = torch.zeros((3,3)).fill_(1/3)
  gt_output = torch.zeros((3,3,3))
  assert (torch.all(torch.lt(torch.abs(torch.add(alpha, -gt_alpha1)), 1e-4))) == 1
  assert (torch.all(torch.lt(torch.abs(torch.add(attention_context, -gt_output)), 1e-4))) == 1
  test_encoder_out = torch.tensor([[[1.0,1.0,1.0],[-1.0,-1.0,-1.0],[0.0,0.0,0.0]]])
  test_decoder_hidden = torch.tensor([[1.0,1.0,1.0]])
  attention_context2, alpha2 = model(test_encoder_out, test_encoder_out, test_decoder_hidden)
  gt_alpha2 = torch.tensor([[0.8668, 0.0159, 0.1173]]).float()
  gt_context2 = torch.tensor([[0.8509,0.8509,0.8509]]).float()
  assert (torch.all(torch.lt(torch.abs(torch.add(alpha2, -gt_alpha2)), 1e-4))) == 1
  assert (torch.all(torch.lt(torch.abs(torch.add(attention_context2, -gt_context2)), 1e-4))) == 1
  print ("PASS! Good job! ")

In [7]:
## Test code  Error 나면 코드 확인필요 "PASS! Good job" 이 아무문제 없이 출력시 성공 
test_attention_general()

PASS! Good job! 
